In [ ]:
# Imports
import kagglehub
import pandas as pd
import numpy as np
import kagglehub
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from kagglehub import KaggleDatasetAdapter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Load the dataset
#df = pd.read_csv("drive/MyDrive/McDonald_s_Reviews.csv", encoding="latin1")
df = pd.read_csv("McDonald_s_Reviews.csv", encoding="latin1")
print(df.head())
print("\nShape: ", df.shape)

df.columns = df.columns.str.strip()
print("Missing values before handling:")
print(df.isnull().sum())

# Handle missing values numerical columns with median
df['latitude'].fillna(df['latitude'].mean(), inplace=True)
df['longitude'].fillna(df['longitude'].mean(), inplace=True)

print("\nMissing values after handling:")
print(df.isnull().sum())

   reviewer_id  store_name              category  \
0            1  McDonald's  Fast food restaurant   
1            2  McDonald's  Fast food restaurant   
2            3  McDonald's  Fast food restaurant   
3            4  McDonald's  Fast food restaurant   
4            5  McDonald's  Fast food restaurant   

                                       store_address  latitude   longitude  \
0  13749 US-183 Hwy, Austin, TX 78750, United States  30.460718 -97.792874   
1  13749 US-183 Hwy, Austin, TX 78750, United States  30.460718 -97.792874   
2  13749 US-183 Hwy, Austin, TX 78750, United States  30.460718 -97.792874   
3  13749 US-183 Hwy, Austin, TX 78750, United States  30.460718 -97.792874   
4  13749 US-183 Hwy, Austin, TX 78750, United States  30.460718 -97.792874   

  rating_count   review_time  \
0        1,240  3 months ago   
1        1,240    5 days ago   
2        1,240    5 days ago   
3        1,240   a month ago   
4        1,240  2 months ago   

                         

<ipython-input-4-d7cd96d2ead9>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['latitude'].fillna(df['latitude'].mean(), inplace=True)
<ipython-input-4-d7cd96d2ead9>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

In [ ]:
# Removing unnecessary features
df = df.drop("reviewer_id", axis=1)
df = df.drop("store_name", axis=1)
df = df.drop("category", axis=1)
df.head()

,store_address,latitude,longitude,rating_count,review_time,review,rating
0,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


In [ ]:
# Encode rating column
df['rating'] = df['rating'].astype(str).str.extract(r'(\d)').astype(int)
df['rating'] = df['rating'].map(lambda x: -1 if x <= 2 else (0 if x == 3 else 1))

#Encode rating_count
df['rating_count'] = df['rating_count'].str.replace(',', '').astype(int)

#Encode store_address
from sklearn.preprocessing import LabelEncoder

#Encode store_address
le = LabelEncoder()
df['store_address'] = le.fit_transform(df['store_address'])

#Encode review_time
import re

def convert_review_time(time_str):
    if pd.isna(time_str):  # Handle missing values
        return None

    match = re.search(r'\d+', str(time_str))  # Ensure it's a string
    if not match:
        return None  # If no number is found, return None

    num = int(match.group())  # Extract number

    if "day" in time_str:
        return num
    elif "month" in time_str:
        return num * 30
    elif "year" in time_str:
        return num * 365
    else:
        return None  # Catch unexpected cases

df['review_time'] = df['review_time'].apply(convert_review_time)
df.head()

,store_address,latitude,longitude,rating_count,review_time,review,rating
0,8,30.460718,-97.792874,1240,90.0,Why does it look like someone spit on my food?...,-1
1,8,30.460718,-97.792874,1240,5.0,It'd McDonalds. It is what it is as far as the...,1
2,8,30.460718,-97.792874,1240,5.0,Made a mobile order got to the speaker and che...,-1
3,8,30.460718,-97.792874,1240,NaN,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,1
4,8,30.460718,-97.792874,1240,60.0,"I repeat my order 3 times in the drive thru, a...",-1


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=100)  # Limit to 100 features

# Transform the text column
tfidf_matrix = vectorizer.fit_transform(df['review'].fillna(""))  # Convert NaN to empty strings

# Convert sparse matrix to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Merge with original dataset (drop original review column)
df = pd.concat([df.drop(columns=['review']), tfidf_df], axis=1)
df.head()

,store_address,latitude,longitude,rating_count,review_time,rating,10,20,area,ask,...,waiting,want,way,went,window,work,worst,wrong,½s,½ï
0,8,30.460718,-97.792874,1240,90.0,-1,0.0,0.0,0.0,0.0,...,0.0,0.527387,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,8,30.460718,-97.792874,1240,5.0,1,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
2,8,30.460718,-97.792874,1240,5.0,-1,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.048056,0.0,0.053121,0.0,0.000000,0.0,0.986156
3,8,30.460718,-97.792874,1240,NaN,1,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.997586
4,8,30.460718,-97.792874,1240,60.0,-1,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.283198,0.0,0.263631,0.0,0.000000


In [ ]:
# Removes review column and adds these columns
tfidf_df.columns

Index(['10', '20', 'area', 'ask', 'asked', 'bad', 'best', 'better', 'big',
       'breakfast', 'burger', 'busy', 'came', 'chicken', 'clean', 'coffee',
       'cold', 'come', 'customer', 'customers', 'day', 'did', 'didn', 'dirty',
       'don', 'drive', 'eat', 'employees', 'excellent', 'experience', 'fast',
       'food', 'fresh', 'friendly', 'fries', 'gave', 'going', 'good', 'got',
       'great', 'homeless', 'horrible', 'hot', 'ice', 'inside', 'just', 'kids',
       'know', 'like', 'line', 'location', 'long', 'lot', 'love', 'make',
       'manager', 'mcdonald', 'mcdonalds', 'meal', 'minutes', 'need',
       'neutral', 'new', 'nice', 'night', 'open', 'order', 'ordered', 'orders',
       'people', 'place', 'poor', 'quick', 'really', 'restaurant', 'right',
       'rude', 'said', 'say', 'service', 'slow', 'staff', 'terrible', 'time',
       'times', 'told', 'took', 've', 'wait', 'waited', 'waiting', 'want',
       'way', 'went', 'window', 'work', 'worst', 'wrong', '½s', '½ï'],
      dtype

In [ ]:
# Drop columns that are completely empty
X = df.drop(columns=['rating'])
X = X.dropna(axis=1, how='all')  # Drop columns where all values are NaN

# Apply imputation
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Convert back to DataFrame
df_imputed = pd.DataFrame(X_imputed, columns=X.columns)

# SelectKBest for feature selection
selector = SelectKBest(score_func=f_classif, k=99)
X_selected = selector.fit_transform(df_imputed, df['rating'])
selected_features = X.columns[selector.get_support()]

# Convert selected features into DataFrame
df_selected = pd.DataFrame(X_selected, columns=selected_features)

print(selected_features)


Index(['store_address', 'latitude', 'longitude', 'rating_count', 'review_time',
       '10', '20', 'area', 'ask', 'asked', 'bad', 'best', 'better',
       'breakfast', 'burger', 'busy', 'came', 'chicken', 'clean', 'cold',
       'come', 'customer', 'customers', 'day', 'did', 'didn', 'dirty', 'don',
       'drive', 'eat', 'employees', 'excellent', 'experience', 'fast', 'food',
       'fresh', 'friendly', 'fries', 'gave', 'going', 'good', 'got', 'great',
       'homeless', 'horrible', 'hot', 'ice', 'inside', 'just', 'kids', 'know',
       'like', 'line', 'location', 'long', 'lot', 'love', 'make', 'manager',
       'mcdonald', 'meal', 'minutes', 'need', 'neutral', 'nice', 'night',
       'open', 'order', 'ordered', 'orders', 'people', 'place', 'poor',
       'quick', 'right', 'rude', 'said', 'say', 'service', 'slow', 'staff',
       'terrible', 'time', 'times', 'told', 'took', 've', 'wait', 'waited',
       'waiting', 'want', 'way', 'went', 'window', 'work', 'worst', 'wrong',
       '½s',

In [ ]:
# Define feature matrix (X) and target variable (y)
X = df_selected  # Features from previous feature selection step
y = df['rating']  # Target variable

# Apply RandomOverSampler to balance dataset
oversampler = RandomOverSampler(random_state=42)
X_balanced, y_balanced = oversampler.fit_resample(X, y)

# Convert back to DataFrame
df_balanced = pd.DataFrame(X_balanced, columns=X.columns)
df_balanced['rating'] = y_balanced  # Add the target column back

print(df_balanced['rating'].value_counts())  # Check class distribution


rating
-1    16061
 1    16061
 0    16061
Name: count, dtype: int64


In [ ]:
# Split the Data
X = df_selected
y = df['rating']
X_train, X_temp, y_train, y_temp = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Re-apply imputation AFTER splitting
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_val = imputer.transform(X_val)
X_test = imputer.transform(X_test)

print(f"Train set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")

Train set: (33728, 99), (33728,)
Validation set: (7227, 99), (7227,)
Test set: (7228, 99), (7228,)


In [ ]:
rf_model = RandomForestClassifier(n_estimators=10, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
print(f"Random Forest")

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, cohen_kappa_score, classification_report
)
from sklearn.preprocessing import label_binarize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Compute Performance Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
mcc = matthews_corrcoef(y_test, y_pred)
cohen_kappa = cohen_kappa_score(y_test, y_pred)

# Multi-class ROC-AUC Score (Fixed)
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))  # Convert to one-hot encoding
roc_auc = roc_auc_score(y_test_bin, rf_model.predict_proba(X_test), multi_class="ovr")

# 4️⃣ Print Metrics
print("\n🔹 Classification Report:\n", classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"MCC: {mcc:.4f}")
print(f"Cohen's Kappa: {cohen_kappa:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Random Forest

🔹 Classification Report:
               precision    recall  f1-score   support

          -1       0.84      0.86      0.85      2388
           0       0.84      0.92      0.88      2464
           1       0.87      0.76      0.81      2376

    accuracy                           0.85      7228
   macro avg       0.85      0.85      0.84      7228
weighted avg       0.85      0.85      0.84      7228

Accuracy: 0.8462
Precision: 0.8472
Recall: 0.8462
F1-score: 0.8450
MCC: 0.7706
Cohen's Kappa: 0.7691
ROC-AUC Score: 0.9524

Confusion Matrix:
[[2051  181  156]
 [  84 2255  125]
 [ 312  254 1810]]


In [ ]:
from sklearn.decomposition import PCA

# Apply PCA to reduce dimensions while keeping 95% variance
pca = PCA(n_components=30)  # Adjust components as needed
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

print(f"Original Features: {X_train.shape[1]}")
print(f"Reduced Features (PCA): {X_train_pca.shape[1]}")

Original Features: 99
Reduced Features (PCA): 30


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import randint
from sklearn.metrics import accuracy_score, matthews_corrcoef, cohen_kappa_score
import numpy as np
import pandas as pd

# Define improved hyperparameter grid
param_dist = {
    "n_estimators": randint(75, 150),  # Increase number of trees
    "max_depth": [None, 10, 20, 30],  # Allow deeper trees
    "min_samples_split": randint(5, 15),  # Reduce risk of overfitting
    "min_samples_leaf": randint(2, 6),
    "bootstrap": [True, False]
}

# Initialize model
rf = RandomForestClassifier(random_state=42)

# Optimized RandomizedSearchCV with more iterations
random_search = RandomizedSearchCV(
    rf, param_distributions=param_dist,
    n_iter=20,  # Explore more
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42,
    return_train_score=True
)

# Fit model
random_search.fit(X_train_pca, y_train)

# Convert results to DataFrame
results_df = pd.DataFrame(random_search.cv_results_)

# Select top 3 best hyperparameters based on accuracy
top_3 = results_df.nlargest(3, 'mean_test_score')

print("\nTop 3 Best Hyperparameter Sets:")
print(top_3[['param_n_estimators', 'param_max_depth', 'param_min_samples_split', 'param_min_samples_leaf', 'param_bootstrap', 'mean_test_score']])

# Evaluate each of the top 3 models
for i, row in top_3.iterrows():
    best_params = {
        "n_estimators": row['param_n_estimators'],
        "max_depth": row['param_max_depth'],
        "min_samples_split": row['param_min_samples_split'],
        "min_samples_leaf": row['param_min_samples_leaf'],
        "bootstrap": row['param_bootstrap']
    }

    # Train a model with the best parameters
    best_rf = RandomForestClassifier(**best_params, random_state=42)
    best_rf.fit(X_train_pca, y_train)

    # Predict on validation set
    y_pred = best_rf.predict(X_val_pca)  # Assuming you have X_val_pca, y_val for validation

    # Calculate metrics
    acc = accuracy_score(y_val, y_pred)
    mcc = matthews_corrcoef(y_val, y_pred)
    kappa = cohen_kappa_score(y_val, y_pred)

    print(f"\nModel {i+1} Performance:")
    print(f"Accuracy: {acc:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"Kappa Score: {kappa:.4f}")


Top 3 Best Hyperparameter Sets:
    param_n_estimators param_max_depth  param_min_samples_split  \
8                  125              30                        7   
15                 118              30                        6   
11                 134            None                        8   

    param_min_samples_leaf  param_bootstrap  mean_test_score  
8                        2            False         0.834885  
15                       3            False         0.834322  
11                       3            False         0.833491  

Model 9 Performance:
Accuracy: 0.8637
MCC: 0.7964
Kappa Score: 0.7956

Model 16 Performance:
Accuracy: 0.8612
MCC: 0.7927
Kappa Score: 0.7919

Model 12 Performance:
Accuracy: 0.8607
MCC: 0.7918
Kappa Score: 0.7910


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, matthews_corrcoef, cohen_kappa_score
import pandas as pd

# Define hyperparameter grid
grid_param = {
    "n_estimators": [50, 100],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 3],
    "bootstrap": [True, False]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    grid_param, cv=3, scoring="accuracy", n_jobs=-1
)

# Fit model
grid_search.fit(X_train_pca, y_train)

# Convert results to DataFrame
results_df = pd.DataFrame(grid_search.cv_results_)

# Select top 3 best hyperparameter sets based on mean test score
top_3 = results_df.nlargest(3, 'mean_test_score')

print("\nTop 3 Best Hyperparameter Sets:")
print(top_3[['param_n_estimators', 'param_max_depth', 'param_min_samples_split', 'param_min_samples_leaf', 'param_bootstrap', 'mean_test_score']])

# Evaluate each of the top 3 models
for i, row in top_3.iterrows():
    best_params = {
        "n_estimators": row['param_n_estimators'],
        "max_depth": row['param_max_depth'],
        "min_samples_split": row['param_min_samples_split'],
        "min_samples_leaf": row['param_min_samples_leaf'],
        "bootstrap": row['param_bootstrap']
    }

    # Train a model with the best parameters
    best_rf = RandomForestClassifier(**best_params, random_state=42)
    best_rf.fit(X_train_pca, y_train)

    # Predict on validation set
    y_pred = best_rf.predict(X_val_pca)  # Assuming you have X_val_pca, y_val for validation

    # Calculate metrics
    acc = accuracy_score(y_val, y_pred)
    mcc = matthews_corrcoef(y_val, y_pred)
    kappa = cohen_kappa_score(y_val, y_pred)

    print(f"\nModel {i+1} Performance:")
    print(f"Accuracy: {acc:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"Kappa Score: {kappa:.4f}")


Top 3 Best Hyperparameter Sets:
    param_n_estimators param_max_depth  param_min_samples_split  \
41                 100              20                        2   
27                 100            None                        5   
29                 100            None                        2   

    param_min_samples_leaf  param_bootstrap  mean_test_score  
41                       1            False         0.835389  
27                       1            False         0.834381  
29                       3            False         0.833788  

Model 42 Performance:
Accuracy: 0.8596
MCC: 0.7908
Kappa Score: 0.7894

Model 28 Performance:
Accuracy: 0.8615
MCC: 0.7931
Kappa Score: 0.7923

Model 30 Performance:
Accuracy: 0.8625
MCC: 0.7945
Kappa Score: 0.7937


In [ ]:
from sklearn.linear_model import LogisticRegression

logreg_model = LogisticRegression(max_iter=10000, random_state=42)
logreg_model.fit(X_train, y_train)
y_pred = logreg_model.predict(X_test)
print(f"Logistic Regression Accuracy: {accuracy}")

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predictions
y_pred = logreg_model.predict(X_test)
print(f"Logistic Regression")

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

precision = precision_score(y_test, y_pred, average='weighted')
print(f"Precision: {precision:.4f}")

mcc = matthews_corrcoef(y_test, y_pred)
print(f"MCC: {mcc:.4f}")

cohen_kappa = cohen_kappa_score(y_test, y_pred)
print(f"Kappa score: {cohen_kappa:.4f}")

# Classification Report (Precision, Recall, F1-score)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Logistic Regression Accuracy: 0.8461538461538461
Logistic Regression
Accuracy: 0.6772
Precision: 0.6791
MCC: 0.5171
Kappa score: 0.5158

Classification Report:
              precision    recall  f1-score   support

          -1       0.68      0.77      0.72      2388
           0       0.60      0.58      0.59      2464
           1       0.76      0.68      0.72      2376

    accuracy                           0.68      7228
   macro avg       0.68      0.68      0.68      7228
weighted avg       0.68      0.68      0.68      7228


Confusion Matrix:
[[1841  450   97]
 [ 605 1439  420]
 [ 269  492 1615]]


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, matthews_corrcoef, cohen_kappa_score, classification_report, confusion_matrix

# Standardizing the features before applying PCA
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit only on training data
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Apply PCA to retain 95% variance
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_val_pca = pca.transform(X_val_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Number of features before and after PCA
print(f"Original number of features: {X_train.shape[1]}")
print(f"Reduced number of features after PCA: {X_train_pca.shape[1]}")

# Define parameter distribution for Logistic Regression with appropriate solver-penalty pairs
param_dist = {
    'C': [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs'],
    'max_iter': [200, 500, 1000],
    'penalty': ['l1', 'l2']
}

# For lbfgs solver, restrict to 'l2' penalty only (no 'l1')
for idx, solver in enumerate(param_dist['solver']):
    if solver == 'lbfgs':
        param_dist['penalty'] = ['l2']  # Set penalty to 'l2' for lbfgs solver

# Perform RandomizedSearchCV to tune hyperparameters
random_search = RandomizedSearchCV(
    LogisticRegression(), param_distributions=param_dist,
    n_iter=10, scoring='accuracy', cv=5, n_jobs=-1, random_state=42
)

# Fit the RandomizedSearchCV on PCA-transformed training data
random_search.fit(X_train_pca, y_train)

# Get top 3 parameter combinations
top_params = random_search.best_params_
print(f"Best Parameters from RandomizedSearchCV: {top_params}")

# Get the top 3 valid parameter sets from the RandomizedSearchCV results
valid_results = [(params, score) for params, score in zip(random_search.cv_results_['params'], random_search.cv_results_['mean_test_score']) if not np.isnan(score)]
sorted_valid_results = sorted(valid_results, key=lambda x: x[1], reverse=True)

# Top 3 valid parameter combinations
top_3_params = [result[0] for result in sorted_valid_results[:3]]
top_3_scores = [result[1] for result in sorted_valid_results[:3]]

print(f"\nTop 3 Valid Parameter Combinations:")
for idx, params in enumerate(top_3_params, start=1):
    print(f"Top {idx} Params: {params} - Accuracy: {top_3_scores[idx-1]:.4f}")

# Now, let's train and evaluate the models with the top 3 sets of parameters
for idx, params in enumerate(top_3_params, start=1):
    print(f"\nTraining and Evaluating Model {idx} with Parameters: {params}")

    # Create and train the Logistic Regression model with current best parameters
    logreg = LogisticRegression(
        C=params['C'],
        solver=params['solver'],
        max_iter=params['max_iter'],
        penalty=params['penalty']
    )
    logreg.fit(X_train_pca, y_train)

    # Predict on Validation and Test Sets
    y_val_pred = logreg.predict(X_val_pca)
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, average='weighted')
    mcc = matthews_corrcoef(y_val, y_val_pred)
    kappa = cohen_kappa_score(y_val, y_val_pred)

    print(f"🔹 Validation Accuracy: {accuracy:.4f}")
    print(f"🔹 Validation Precision: {precision:.4f}")
    print(f"🔹 Validation MCC: {mcc:.4f}")
    print(f"🔹 Validation Kappa Score: {kappa:.4f}")

    print("\n🔹 Classification Report:\n", classification_report(y_val, y_val_pred))
    print("\n🔹 Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred))

    # Predict on Test Set
    y_test_pred = logreg.predict(X_test_pca)
    accuracy = accuracy_score(y_test, y_test_pred)
    precision = precision_score(y_test, y_test_pred, average='weighted')
    mcc = matthews_corrcoef(y_test, y_test_pred)
    kappa = cohen_kappa_score(y_test, y_test_pred)

    print(f"✅ Test Accuracy: {accuracy:.4f}")
    print(f"✅ Test Precision: {precision:.4f}")
    print(f"✅ Test MCC: {mcc:.4f}")
    print(f"✅ Test Kappa Score: {kappa:.4f}")

    print("\n✅ Test Classification Report:\n", classification_report(y_test, y_test_pred))
    print("\n✅ Test Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))


Original Features: 99
Reduced Features (PCA): 30


In [ ]:
# Basic KNN Implementation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, cohen_kappa_score, classification_report
)
from sklearn.preprocessing import label_binarize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize and train basic KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)
print(f"KNN Model")

# Compute Performance Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
mcc = matthews_corrcoef(y_test, y_pred)
cohen_kappa = cohen_kappa_score(y_test, y_pred)

# Multi-class ROC-AUC Score
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))  # Convert to one-hot encoding
roc_auc = roc_auc_score(y_test_bin, knn_model.predict_proba(X_test), multi_class="ovr")

print("\n🔹 Classification Report:\n", classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"MCC: {mcc:.4f}")
print(f"Cohen's Kappa: {cohen_kappa:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


In [ ]:

# Apply PCA for dimensionality reduction
from sklearn.decomposition import PCA

# Apply PCA to reduce dimensions while keeping 95% variance
pca = PCA(n_components=30)  # Adjust components as needed
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

print(f"Original Features: {X_train.shape[1]}")
print(f"Reduced Features (PCA): {X_train_pca.shape[1]}")



In [ ]:
# Randomized Search - 3 best parameter combinations
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import randint
import numpy as np
import pandas as pd

# Define improved hyperparameter grid for KNN
param_dist = {
    "n_neighbors": randint(3, 20),
    "weights": ["uniform", "distance"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "p": [1, 2],  # 1=Manhattan, 2=Euclidean
    "leaf_size": randint(10, 50)
}

# Initialize model
knn = KNeighborsClassifier()

# Optimized RandomizedSearchCV with more iterations
random_search = RandomizedSearchCV(
    knn, param_distributions=param_dist,
    n_iter=20,  # Explore more
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42,
    return_train_score=True
)

# Fit model
random_search.fit(X_train_pca, y_train)

# Convert results to DataFrame
results_df = pd.DataFrame(random_search.cv_results_)

# Select top 3 best hyperparameters based on accuracy
top_3 = results_df.nlargest(3, 'mean_test_score')

print("\nTop 3 Best Hyperparameter Sets:")
print(top_3[['param_n_neighbors', 'param_weights', 'param_algorithm', 'param_p', 'param_leaf_size', 'mean_test_score']])

# Evaluate each of the top 3 models
for i, row in top_3.iterrows():
    best_params = {
        "n_neighbors": row['param_n_neighbors'],
        "weights": row['param_weights'],
        "algorithm": row['param_algorithm'],
        "p": row['param_p'],
        "leaf_size": row['param_leaf_size']
    }

    # Train a model with the best parameters
    best_knn = KNeighborsClassifier(**best_params)
    best_knn.fit(X_train_pca, y_train)

    # Predict on validation set
    y_pred = best_knn.predict(X_val_pca)

    # Calculate metrics
    acc = accuracy_score(y_val, y_pred)
    mcc = matthews_corrcoef(y_val, y_pred)
    kappa = cohen_kappa_score(y_val, y_pred)

    print(f"\nModel {i+1} Performance:")
    print(f"Accuracy: {acc:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"Kappa Score: {kappa:.4f}")



In [ ]:
# Grid Search - Top 3 parameter combinations
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, matthews_corrcoef, cohen_kappa_score
import pandas as pd

# Define hyperparameter grid for KNN
grid_param = {
    "n_neighbors": [3, 5, 7, 9],
    "weights": ["uniform", "distance"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "p": [1, 2],
    "leaf_size": [20, 30, 40]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    KNeighborsClassifier(),
    grid_param, cv=3, scoring="accuracy", n_jobs=-1
)

# Fit model
grid_search.fit(X_train_pca, y_train)

# Convert results to DataFrame
results_df = pd.DataFrame(grid_search.cv_results_)

# Select top 3 best hyperparameter sets based on mean test score
top_3 = results_df.nlargest(3, 'mean_test_score')

print("\nTop 3 Best Hyperparameter Sets:")
print(top_3[['param_n_neighbors', 'param_weights', 'param_algorithm', 'param_p', 'param_leaf_size', 'mean_test_score']])

# Evaluate each of the top 3 models
for i, row in top_3.iterrows():
    best_params = {
        "n_neighbors": row['param_n_neighbors'],
        "weights": row['param_weights'],
        "algorithm": row['param_algorithm'],
        "p": row['param_p'],
        "leaf_size": row['param_leaf_size']
    }

    # Train a model with the best parameters
    best_knn = KNeighborsClassifier(**best_params)
    best_knn.fit(X_train_pca, y_train)

    # Predict on validation set
    y_pred = best_knn.predict(X_val_pca)

    # Calculate metrics
    acc = accuracy_score(y_val, y_pred)
    mcc = matthews_corrcoef(y_val, y_pred)
    kappa = cohen_kappa_score(y_val, y_pred)

    print(f"\nModel {i+1} Performance:")
    print(f"Accuracy: {acc:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"Kappa Score: {kappa:.4f}")
